In [ ]:
from __init__ import *
from adapt_pcr_external_tools import *
from primer_modules import *
from ml_modules import *

import numpy as np
import pandas as pd
import time
import joblib
import gzip
from multiprocessing import Pool, cpu_count
from tqdm.auto import tqdm
from collections import defaultdict, Counter
from Bio import SeqIO, Seq, Entrez
from io import StringIO
from Levenshtein import distance

# Setting

In [ ]:
from Bio.SeqUtils import MeltingTemp, gc_fraction
def get_tm(seq):
    return MeltingTemp.Tm_NN(seq, Na=50, Mg=1.5, dNTPs=.6)

def get_dg_vienna(seq1, seq2):
    inp = '"%s\n%s"'%(seq1,seq2)
    res = !echo -e $inp | $RNAduplex - --noconv --paramFile=DNA 2>tmp
    dg = float(res[0].split()[-1][1:-1])
    return dg

Entrez.email = "bsc0371@gmail.com"
def get_cdna_sequence(nm_id):
    handle = Entrez.efetch(db="nucleotide", id=nm_id, rettype="fasta", retmode="text")
    fasta_data = handle.read()
    handle.close()
    
    record = SeqIO.read(StringIO(fasta_data), "fasta")
    return record.description, str(record.seq)

In [ ]:
#ML = '/home/jupyter/ADAPT_PCR_share/safe/resources/ml/0725_combined_model.pth'

ML = '/home/jupyter/ADAPT_PCR_share/safe/resources/ml/0725_combined_model.pth'
SCALER = '/home/jupyter/ADAPT_PCR_share/safe/resources/ml/0728_scaler.joblib'
FEATS = ['f_len','f_Tm','f_GC','f_indel','f_mm','r_len','r_Tm','r_GC','r_indel','r_mm','prod_len','prod_Tm']

scaler = joblib.load(SCALER)
model = torch.load(ML, weights_only=False)
model.eval()

In [ ]:
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size, bidirectional=False):
        super(LSTMModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.bidirectional = bidirectional
        self.num_directions = 2 if bidirectional else 1

        self.lstm = nn.LSTM(
            input_size,
            hidden_size,
            num_layers,
            batch_first=True,
            bidirectional=bidirectional
        )

        # Fully connected layer to change shape into output size
        self.fc = nn.Linear(hidden_size * self.num_directions, output_size)
        self.sigout = nn.Sigmoid()

    def forward(self, x):
        device = x.device

        h0 = torch.zeros(self.num_layers * self.num_directions, x.size(0), self.hidden_size).to(device)
        c0 = torch.zeros(self.num_layers * self.num_directions, x.size(0), self.hidden_size).to(device)

        out, _ = self.lstm(x, (h0, c0))  # out: (batch_size, seq_len, hidden_size * num_directions)

        out = self.fc(out[:, -1, :])  # Last time step
        out = self.sigout(out)
        return out

import pickle
classifier_path = '/home/jupyter/ADAPT_PCR_share/safe/resources/ml/0721_LSTM_Classifier.pkl'
classifier = pickle.load(open(classifier_path, 'rb'))
classifier.lstm.flatten_parameters()

In [ ]:
def one_hot_encode_pbs_gap(df_seqs):
    primer_encoded = []
    target_encoded = []
    for i,row in df_seqs.iterrows():
        fenc, ftenc, renc, rtenc = row[['pseq_f','tseq_f','pseq_r','tseq_r']].apply(one_hot_encode)
        prienc = np.append(fenc,renc,axis=0)
        tarenc = np.append(ftenc,rtenc,axis=0)
        primer_encoded.append(prienc)
        target_encoded.append(tarenc)
    primer_encoded = np.array(primer_encoded)
    target_encoded = np.array(target_encoded)
    final_encoded = np.append(target_encoded, primer_encoded, axis=2)
    print(final_encoded.shape)
    return torch.tensor(final_encoded, dtype=torch.float32)

In [ ]:
# genef1 = "/home/jupyter/ADAPT_PCR_share/safe/resources/genomes/human_all.rna.fna.gz"
# genef2 = "/home/jupyter/ADAPT_PCR_share/safe/resources/genomes/refMrna.fa.gz"
# nmseqs = { s.id.split('.')[0]:str(s.seq) for s in SeqIO.parse(gzip.open(genef1,'rt'), "fasta") }
# print('# NMs in f1:', len(nmseqs))
# for s in SeqIO.parse(gzip.open(genef2,'rt'), "fasta"):
#     try:
#         seq = nmseqs[s.id]
#     except KeyError:
#         nmseqs[s.id] = str(s.seq).upper()
# print('# NMs in f1 & f2:', len(nmseqs))

# prinms = set(oritbl['NM'].tolist())
# tseqs = {}
# for nm in prinms:
#     if nm not in allnms:
#         tseqs[nm] = get_cdna_sequence(nm)[1]

# genef3 = "/home/jupyter/ADAPT_PCR_share/safe/resources/genomes/Origene_missing.fa"
# with open(genef3, 'wt') as out:
#     for nm in sorted(tseqs.keys()):
#         out.write('>%s\n%s\n' % (nm, tseqs[nm].upper()))

# genef3 = "/home/jupyter/ADAPT_PCR_share/safe/resources/genomes/Origene_missing.fa"
# for s in SeqIO.parse(genef3, "fasta"):
#     nmseqs[s.id] = str(s.seq)
# print('# NMs in f1, f2 & f3:', len(nmseqs))

genef_comb = "/home/jupyter/ADAPT_PCR_share/safe/resources/genomes/nm_combined.fa"
# with open(genef_comb, 'wt') as out:
#     for nm in sorted(nmseqs.keys()):
#         out.write('>%s\n%s\n' % (nm, nmseqs[nm].upper()))

nmseqs = {}
for s in SeqIO.parse(genef_comb, "fasta"):
    nmseqs[s.id] = str(s.seq)
print('# NMs:', len(nmseqs))

# Origene

## True sets

In [ ]:
allnms = set(nmseqs.keys())
pseqf = '/home/jupyter/ADAPT_PCR_share/safe/resources/origene_primers_0731.csv'
tmptbl = pd.read_csv(pseqf).drop_duplicates('NM').dropna()
oritbl = tmptbl[tmptbl['NM'].apply(lambda nm: nm.startswith('NM') and nm in allnms)].drop_duplicates('NM')
print('# primers:', len(tmptbl))
print('# target seq prepared:', len(oritbl))
oritbl.head(1)

In [ ]:
import random
cols = ['Gene', 'tname', 'pseq_f', 'pseq_r']
def find_match(seq, ref, dcut=5):
    if len(ref) < len(seq):
        return (-1, 999) 
    if seq in ref:
        return (ref.find(seq), 0)
    sortvs = sorted([(distance(seq, ref[i:i+len(seq)]), i) for i in range(len(ref)-len(seq))])
    d, st = sortvs[0]
    return (st, d) if d <= dcut else (-1, 999)

def prepare_input_parallel(df, nmseqs, parse_func):
    rows = []
    for row in df.to_dict('records'):
        row['tseq'] = nmseqs[row['tname']]
        rows.append(row)
        
    with Pool(processes=cpu_count()) as pool:
        results = tqdm(pool.map(parse_func, rows), total=len(rows))

    filtered_rows = [row for row in results if row is not None]
    return pd.DataFrame(filtered_rows)

def prepare_input(row):
    gene, nm, fseq, rseq, tseq = row['Gene'], row['tname'], row['pseq_f'], row['pseq_r'], row['tseq']
    
    fst, fd = find_match(fseq, tseq)
    rst, rd = find_match(reverse_complement_dna(rseq), tseq[fst:])
    rst = fst + rst
    
    if 0 <= fst < rst:
        row['tseq_f'] = tseq[fst:fst + len(fseq)]
        row['tseq_r'] = reverse_complement_dna(tseq[rst:rst + len(rseq)])
        prod = tseq[fst:rst + len(rseq)]
        row['prod_len'] = len(prod)
        row['prod_Tm'] = get_tm(prod)
        
        for prefix, pseq, dist in zip(['f', 'r'], [fseq, rseq], [fd, rd]):
            row[f'{prefix}_len'] = len(pseq)
            row[f'{prefix}_GC'] = gc_fraction(pseq)
            row[f'{prefix}_Tm'] = get_tm(pseq)
            row[f'{prefix}_indel'] = 0
            row[f'{prefix}_mm'] = dist

        return row
    return None

def mut_nuc(seq):
    mutd = {'A':'G','G':'A','T':'C','C':'T'}
    return ''.join([mutd[nuc] for nuc in seq])

def introduce_5p_mut(seq, mutl):
    #return mut_nuc(seq[:mutl]) + seq[mutl:]
    return reverse_complement_dna(seq[:mutl]) + seq[mutl:]

def introduce_3p_mut(seq, mutl):
    #return mut_nuc(seq[:mutl]) + seq[mutl:]
    return seq[:-mutl] + reverse_complement_dna(seq[-mutl:])

def prepare_input_5p_mut(row, mutcnts=range(1,9), weights=[1,2,3,4,4,3,2,1]):
    gene, nm, fseq, rseq, tseq = row['Gene'], row['tname'], row['pseq_f'], row['pseq_r'], row['tseq']
    
    fst, fd = find_match(fseq, tseq)
    rst, rd = find_match(reverse_complement_dna(rseq), tseq[fst:])
    rst = fst + rst
        
    if 0 <= fst < rst:
        fmut = random.choices(mutcnts, weights=weights, k=1)[0]
        rmut = random.choices(mutcnts, weights=weights, k=1)[0]
        row['tseq_f'] = introduce_5p_mut(tseq[fst:fst + len(fseq)], fmut)
        row['tseq_r'] = introduce_5p_mut(reverse_complement_dna(tseq[rst:rst + len(rseq)]), rmut)
        prod = tseq[fst:rst + len(rseq)]
        row['prod_len'] = len(prod)
        row['prod_Tm'] = get_tm(prod)
    
        for prefix, pseq, dist, mcnt in zip(['f', 'r'], [fseq, rseq], [fd, rd], [fmut, rmut]):
            row[f'{prefix}_len'] = len(pseq)
            row[f'{prefix}_GC'] = gc_fraction(pseq)
            row[f'{prefix}_Tm'] = get_tm(pseq)
            row[f'{prefix}_indel'] = 0
            row[f'{prefix}_mm'] = dist + mcnt

        return row
    return None

def prepare_input_3p_mut(row, mutcnts=range(3,6), weights=[1,1,1]):
    gene, nm, fseq, rseq, tseq = row['Gene'], row['tname'], row['pseq_f'], row['pseq_r'], row['tseq']
    
    fst, fd = find_match(fseq, tseq)
    rst, rd = find_match(reverse_complement_dna(rseq), tseq[fst:])
    rst = fst + rst
        
    if 0 <= fst < rst:
        fmut = random.choices(mutcnts, weights=weights, k=1)[0]
        rmut = random.choices(mutcnts, weights=weights, k=1)[0]
        row['tseq_f'] = introduce_3p_mut(tseq[fst:fst + len(fseq)], fmut)
        row['tseq_r'] = introduce_3p_mut(reverse_complement_dna(tseq[rst:rst + len(rseq)]), rmut)
        prod = tseq[fst:rst + len(rseq)]
        row['prod_len'] = len(prod)
        row['prod_Tm'] = get_tm(prod)
    
        for prefix, pseq, dist, mcnt in zip(['f', 'r'], [fseq, rseq], [fd, rd], [fmut, rmut]):
            row[f'{prefix}_len'] = len(pseq)
            row[f'{prefix}_GC'] = gc_fraction(pseq)
            row[f'{prefix}_Tm'] = get_tm(pseq)
            row[f'{prefix}_indel'] = 0
            row[f'{prefix}_mm'] = dist + mcnt

        return row
    return None

start = time.time()
inp = oritbl.drop(['URL'], axis=1)
inp.columns = cols
inptbl = prepare_input_parallel(inp, nmseqs, prepare_input).dropna()
inptbl_5pmut = prepare_input_parallel(inp, nmseqs, prepare_input_5p_mut).dropna()
inptbl_3pmut = prepare_input_parallel(inp, nmseqs, prepare_input_3p_mut).dropna()
inptbl = pd.concat([inptbl,inptbl_5pmut])
runtime = time.time() - start
print('# valid inputs:', len(inptbl))
print('Run time (parsing): %.3f sec' % runtime)

In [ ]:
def encode_row(row):
    fenc = one_hot_encode(row['pseq_f'])
    ftenc = one_hot_encode(row['tseq_f'])
    renc = one_hot_encode(row['pseq_r'])
    rtenc = one_hot_encode(row['tseq_r'])

    prienc = np.append(fenc, renc, axis=0)   # Primer
    tarenc = np.append(ftenc, rtenc, axis=0) # Target
    combined = np.append(tarenc, prienc, axis=1)
    return combined

def one_hot_encode_pbs_gap_parallel(df_seqs):
    rows = df_seqs.to_dict('records')
    with Pool(processes=cpu_count()) as pool:
        results = pool.map(encode_row, rows)

    final_encoded = np.array(results)  # (batch, 56, 10)
    print(final_encoded.shape)
    return torch.tensor(final_encoded, dtype=torch.float32)

start = time.time()
seq_input = one_hot_encode_pbs_gap_parallel(inptbl)
fea_input = scaler.transform(inptbl[FEATS])
labels = np.array([0]*len(inptbl))
dataset = PcrDataset(seq_input, fea_input, labels)
loader = DataLoader(dataset, batch_size=64, shuffle=False)
runtime = time.time() - start
print('Run time (encoding): %.3f sec' % runtime)

In [ ]:
inpf = '/home/jupyter/ADAPT_PCR_share/safe/design/pipeline/origene_primers_0806.false.csv'

start = time.time()
predict_cls, predict_reg = [], []
with torch.no_grad():
    for seq_in, fea_in, lab in tqdm(loader, desc='Infering'):
        seq_in, fea_in, lab = seq_in.to(device).float(), fea_in.to(device).float(), lab.to(device).float()
        out_cls = classifier(seq_in)
        out_reg = model(fea_in, seq_in)
        if len(seq_in)==1:
            predict_cls.append(np.array([out_cls.squeeze().detach().cpu().numpy()]))
            predict_reg.append(np.array([out_reg.squeeze().detach().cpu().numpy()]))
        else:
            predict_cls.append(out_cls.squeeze().detach().cpu().numpy())
            predict_reg.append(out_reg.squeeze().detach().cpu().numpy())
    predict_cls = np.concatenate(predict_cls)
    predict_reg = np.concatenate(predict_reg)
inptbl.loc[:,'classifier'] = predict_cls
inptbl.loc[:,'regressor'] = predict_reg
inptbl.to_csv(inpf, index=False)
runtime = time.time() - start
print('Run time (inference): %.3f sec' % runtime)

In [ ]:
fig, (ax1,ax2) = plt.subplots(1, 2, figsize=(5,2), sharey=True)
ax1.hist(inptbl['classifier'])
ax2.hist(inptbl['regressor'])

## False sets

In [ ]:
fasta = '/home/jupyter/ADAPT_PCR_share/safe/resources/origene_primers_0731.fa'
# with open(fasta, 'wt') as out:
#     for _, row in inptbl.iterrows():
#         nm = row['tname']
#         fseq = row['pseq_f']
#         rseq = row['pseq_r']
#         out.write('>%s-f\n%s\n>%s-r\n%s\n'%(nm, fseq, nm, rseq))
!wc -l $fasta

### Mapping to viral sequences

In [ ]:
viralf = '/home/jupyter/ADAPT_PCR_share/safe/resources/genomes/human_viral_coding.fa'
viralseqs = {s.id:str(s.seq) for s in SeqIO.parse(viralf, 'fasta')}
print('# viral coding seqs:', len(viralseqs))

In [ ]:
mapopt = '--mp 2,2 --rdg 4,4 --rfg 4,4 -L 8 -N 1 --score-min L,-0.6,-0.6'
threads = 4
ref = '/home/jupyter/ADAPT_PCR_share/safe/resources/genomes/bowtie2/human_viral_coding'
sam = '/home/jupyter/ADAPT_PCR_share/safe/design/pipeline/origene_primers_0731.viral.sam'
multimap = 1000
command = '%s -x %s -U %s -f -p %i -k %i %s --no-hd --no-unal > %s'\
        % (bowtie2, ref, fasta, threads, multimap, mapopt, sam)
#!$command

In [ ]:
parsed = sam.replace('.sam','.parsed')
pseqs = parsed + '.pseq'
tseqs = parsed + '.tseq'
mapped = sam.replace('.sam','.mapped')

comm1 = '%s < %s > %s' % (sam2pairwise, sam, parsed)
comm2 = 'awk "NR %% 4 == 2" %s > %s' % (parsed, pseqs)
comm3 = 'awk "NR %% 4 == 0" %s > %s' % (parsed, tseqs)
comm4 = 'cut -f1-4 %s | paste - %s | paste - %s > %s' % (sam, pseqs, tseqs, mapped)

# !$comm1
# !$comm2
# !$comm3
# !$comm4
# delete([parsed,pseqs,tseqs])

In [ ]:
names = ['pname','orientation','tname','start','pseq','tseq']
subcols = ['pname','start','pseq','tseq','ptarget']
minl, maxl = 50, 10000
def pair_primers(args):
    tname, grp, tseq = args
    #fors = grp.loc[(grp['orientation']==0)&(grp['forrev']=='f'), subcols]
    #revs = grp.loc[(grp['orientation']==16)&(grp['forrev']=='r'), subcols]
    fors = grp.loc[(grp['orientation']==0), subcols]
    revs = grp.loc[(grp['orientation']==16), subcols]
    revs['pseq'] = revs['pseq'].apply(rev_com_enc)
    revs['tseq'] = revs['tseq'].apply(rev_com_enc)
    if fors.empty or revs.empty:
        return None
    
    paired = fors.merge(revs, on='ptarget', suffixes=('_f','_r'))
    if paired.empty:
        return None
    
    paired.loc[:, 'tname'] = tname
    paired.loc[:, 'prod_len'] = paired['start_r'] - paired['start_f'] + paired['pseq_r'].apply(len) + 1
    paired = paired[paired['prod_len'].apply(lambda x: minl <= x <= maxl)]
    if paired.empty:
        return None
    
    paired.loc[:, 'prod_Tm'] = paired.apply(lambda row:
        get_tm(tseq[row['start_f']-1: row['start_r']+len(row['pseq_r'])]), axis=1)

    paired.loc[:, 'f_len'] = paired['pseq_f'].apply(len)
    paired.loc[:, 'f_GC'] = paired['pseq_f'].apply(gc_fraction)
    paired.loc[:, 'f_Tm'] = paired['pseq_f'].apply(get_tm)
    paired.loc[:, 'f_indel'] = paired.apply(lambda row:
        row['pseq_f'].count('-') + row['tseq_f'].count('-'), axis=1)
    paired.loc[:, 'f_mm'] = paired.apply(lambda row:
        sum((c1 != c2) and (c1 != '-') and (c2 != '-') for c1, c2 in zip(row['pseq_f'], row['tseq_f'])), axis=1)

    paired.loc[:, 'r_len'] = paired['pseq_r'].apply(len)
    paired.loc[:, 'r_GC'] = paired['pseq_r'].apply(gc_fraction)
    paired.loc[:, 'r_Tm'] = paired['pseq_r'].apply(get_tm)
    paired.loc[:, 'r_indel'] = paired.apply(lambda row:
        row['pseq_r'].count('-') + row['tseq_r'].count('-'), axis=1)
    paired.loc[:, 'r_mm'] = paired.apply(lambda row:
        sum((c1 != c2) and (c1 != '-') and (c2 != '-') for c1, c2 in zip(row['pseq_r'], row['tseq_r'])), axis=1)

    return paired

# start = time.time()
# samtbl = pd.read_table(mapped, sep='\t', names=names)
# samtbl['orientation'] = samtbl['orientation'].apply(lambda x: x % 256)
# samtbl['ptarget'] = samtbl['pname'].apply(lambda x: x.split('-')[0])
# samtbl['forrev'] = samtbl['pname'].apply(lambda x: x.split('-')[1])
# groups = [ (tname, grp, viralseqs[tname]) for tname, grp in samtbl.groupby('tname') ]
# with Pool(processes=cpu_count()) as pool:
#     results = list(tqdm(pool.map(pair_primers, groups), total=len(groups)))
#     filtered_results = [df for df in results if df is not None]
#     inptbl = pd.concat(filtered_results, ignore_index=True)
# inptbl = inptbl.drop_duplicates(['pseq_f','tseq_f','pseq_r','tseq_r','prod_len'])
# runtime = time.time() - start
# print('# pairs: %i' % len(inptbl))
# print('Run time (parsing): %.3f sec' % runtime)

In [ ]:
def encode_row(row):
    fenc = one_hot_encode(row['pseq_f'])
    ftenc = one_hot_encode(row['tseq_f'])
    renc = one_hot_encode(row['pseq_r'])
    rtenc = one_hot_encode(row['tseq_r'])
    prienc = np.append(fenc, renc, axis=0)   # Primer
    tarenc = np.append(ftenc, rtenc, axis=0) # Target
    combined = np.append(tarenc, prienc, axis=1)
    return combined

def one_hot_encode_pbs_gap_parallel(df_seqs):
    rows = df_seqs.to_dict('records')
    with Pool(processes=cpu_count()) as pool:
        results = pool.map(encode_row, rows)

    final_encoded = np.array(results)  # (batch, 56, 10)
    print(final_encoded.shape)
    return torch.tensor(final_encoded, dtype=torch.float32)

# start = time.time()
# seq_input = one_hot_encode_pbs_gap_parallel(inptbl)
# fea_input = scaler.transform(inptbl[FEATS])
# labels = np.array([0]*len(inptbl))
# dataset = PcrDataset(seq_input, fea_input, labels)
# loader = DataLoader(dataset, batch_size=64, shuffle=False)
# runtime = (time.time() - start)
# print('Run time (encoding): %.3f sec' % runtime)

In [ ]:
inpf = '/home/jupyter/ADAPT_PCR_share/safe/design/pipeline/origene_primers_0731.viral.csv'

start = time.time()
predict_cls, predict_reg = [], []
with torch.no_grad():
    for seq_in, fea_in, lab in tqdm(loader, desc='Infering'):
        seq_in, fea_in, lab = seq_in.to(device).float(), fea_in.to(device).float(), lab.to(device).float()
        out_cls = classifier(seq_in)
        out_reg = model(fea_in, seq_in)
        if len(seq_in)==1:
            predict_cls.append(np.array([out_cls.squeeze().detach().cpu().numpy()]))
            predict_reg.append(np.array([out_reg.squeeze().detach().cpu().numpy()]))
        else:
            predict_cls.append(out_cls.squeeze().detach().cpu().numpy())
            predict_reg.append(out_reg.squeeze().detach().cpu().numpy())
    predict_cls = np.concatenate(predict_cls)
    predict_reg = np.concatenate(predict_reg)
inptbl.loc[:,'classifier'] = predict_cls
inptbl.loc[:,'regressor'] = predict_reg
inptbl.to_csv(inpf, index=False)
runtime = (time.time() - start)
print('Run time (inferece): %.3f sec' % runtime)

In [ ]:
fig, (ax1,ax2) = plt.subplots(1, 2, figsize=(5,2), sharey=True)
ax1.hist(inptbl['classifier'])
ax2.hist(inptbl['regressor'])

### Mapping to human sequences

In [ ]:
# usednmf = '/home/jupyter/ADAPT_PCR_share/safe/resources/origene_primers_nms.fa' 
# with open(usednmf, 'wt') as out:
#     for nm in oritbl['NM']:
#         out.write('>%s\n%s\n' % (nm, nmseqs[nm]))

In [ ]:
mapopt = '--mp 4,4 --rdg 4,4 --rfg 4,4 -L 8 -N 1 --score-min L,-0.6,-0.6'
threads = 4
ref = '/home/jupyter/ADAPT_PCR_share/safe/resources/genomes/bowtie2/origene_primers_nms'
sam = '/home/jupyter/ADAPT_PCR_share/safe/design/pipeline/origene_primers_0806.human.sam'
multimap = 5000
command = '%s -x %s -U %s -f -p %i -k %i %s --no-hd --no-unal > %s'\
           % (bowtie2, ref, fasta, threads, multimap, mapopt, sam)
#!$command

In [ ]:
parsed = sam.replace('.sam','.parsed')
pseqs = parsed + '.pseq'
tseqs = parsed + '.tseq'
mapped = sam.replace('.sam','.mapped')

comm1 = '%s < %s > %s' % (sam2pairwise, sam, parsed)
comm2 = 'awk "NR %% 4 == 2" %s > %s' % (parsed, pseqs)
comm3 = 'awk "NR %% 4 == 0" %s > %s' % (parsed, tseqs)
comm4 = 'cut -f1-4 %s | paste - %s | paste - %s > %s' % (sam, pseqs, tseqs, mapped)

# !$comm1
# !$comm2
# !$comm3
# !$comm4
# delete([parsed,pseqs,tseqs])

In [ ]:
minl, maxl = 50, 10000
names = ['pname','orientation','tname','start','pseq','tseq']
subcols = ['pname','start','pseq','tseq','ptarget']

start = time.time()
samtbl = pd.read_table(mapped, sep='\t', names=names)
samtbl['orientation'] = samtbl['orientation'].apply(lambda x: x % 256)
samtbl['ptarget'] = samtbl['pname'].apply(lambda x: x.split('-')[0])
samtbl['forrev'] = samtbl['pname'].apply(lambda x: x.split('-')[1])
groups = [ (tname, grp, nmseqs[tname]) for tname, grp in samtbl.groupby('tname') ]
with Pool(processes=cpu_count()) as pool:
    results = list(tqdm(pool.map(pair_primers, groups), total=len(groups)))
    filtered_results = [df for df in results if df is not None]
    inptbl = pd.concat(filtered_results, ignore_index=True)
tmptbl = inptbl.drop_duplicates(['pseq_f','tseq_f','pseq_r','tseq_r','prod_len'])

In [ ]:
diff = 100
allts = set(tmptbl['ptarget'])
plens = tmptbl[(tmptbl['ptarget']==tmptbl['tname'])]\
.sort_values('prod_len').drop_duplicates('ptarget')\
.set_index('ptarget')['prod_len'].to_dict()
for target in allts-set(plens.keys()):
    plens[target] = 150
        
inptbl = tmptbl[(tmptbl['ptarget']!=tmptbl['tname'])]
inptbl = inptbl[inptbl.apply(lambda row: row['prod_len']>plens[row['ptarget']]+diff, axis=1)]
runtime = time.time() - start
print('# pairs: %i' % len(inptbl))

In [ ]:
start = time.time()
seq_input = one_hot_encode_pbs_gap_parallel(inptbl)
fea_input = scaler.transform(inptbl[FEATS])
labels = np.array([0]*len(inptbl))
dataset = PcrDataset(seq_input, fea_input, labels)
loader = DataLoader(dataset, batch_size=64, shuffle=False)
runtime = (time.time() - start)
print('Run time (encoding): %.3f sec' % runtime)

In [ ]:
inpf = '/home/jupyter/ADAPT_PCR_share/safe/design/pipeline/origene_primers_0806.human.csv'

start = time.time()
predict_cls, predict_reg = [], []
with torch.no_grad():
    for seq_in, fea_in, lab in tqdm(loader, desc='Infering'):
        seq_in, fea_in, lab = seq_in.to(device).float(), fea_in.to(device).float(), lab.to(device).float()
        out_cls = classifier(seq_in)
        out_reg = model(fea_in, seq_in)
        if len(seq_in)==1:
            predict_cls.append(np.array([out_cls.squeeze().detach().cpu().numpy()]))
            predict_reg.append(np.array([out_reg.squeeze().detach().cpu().numpy()]))
        else:
            predict_cls.append(out_cls.squeeze().detach().cpu().numpy())
            predict_reg.append(out_reg.squeeze().detach().cpu().numpy())
    predict_cls = np.concatenate(predict_cls)
    predict_reg = np.concatenate(predict_reg)
inptbl.loc[:,'classifier'] = predict_cls
inptbl.loc[:,'regressor'] = predict_reg
inptbl.to_csv(inpf, index=False)
runtime = (time.time() - start)
print('Run time (inferece): %.3f sec' % runtime)

## Evaluation

In [ ]:
from sklearn.metrics import roc_auc_score, average_precision_score

In [ ]:
# truef = '/home/jupyter/ADAPT_PCR_share/safe/design/pipeline/origene_primers_0806.true.csv'
# falsef1 = '/home/jupyter/ADAPT_PCR_share/safe/design/pipeline/origene_primers_0806.false.csv'
# falsef2 = '/home/jupyter/ADAPT_PCR_share/safe/design/pipeline/origene_primers_0731.viral.csv'
# falsef3 = '/home/jupyter/ADAPT_PCR_share/safe/design/pipeline/origene_primers_0806.human.csv'
truef = '/home/jupyter/ADAPT_PCR_share/safe/design/pipeline/origene_primers.true.dg.csv'
falsef = '/home/jupyter/ADAPT_PCR_share/safe/design/pipeline/origene_primers.false.dg.csv'
truetbl = pd.read_csv(truef, index_col=0)
falsetbl = pd.read_csv(falsef, index_col=0)
print('# true set: %i, # false set: %i' % (len(truetbl), len(falsetbl)))

In [ ]:
#test_df = pd.read_csv('/home/jupyter/ADAPT_PCR_share/safe/dataset/0717_dataset_test.csv',index_col=[0,1])
truetbl = get_pair_dg_parallel(test_df[test_df['binary']==1])
falsetbl = get_pair_dg_parallel(test_df[test_df['binary']==0])

In [ ]:
truef = '/home/jupyter/ADAPT_PCR_share/safe/design/pipeline/test_set.true.dg.csv'
falsef = '/home/jupyter/ADAPT_PCR_share/safe/design/pipeline/test_set.false.dg.csv'
truetbl.to_csv(truef,index=False)
falsetbl.to_csv(falsef,index=False)

In [ ]:
mmcols = ['f_mm','f_indel','r_mm','r_indel']
dgcols = ['f_dg','r_dg']
for tbl in [truetbl, falsetbl]:
    tbl['mismatches'] = tbl[mmcols].sum(axis=1)
    tbl['dg'] = tbl[dgcols].max(axis=1) 

In [ ]:
scores = defaultdict(dict)
pcols = ['mismatches','dg','classifier','regressor']
plabels = ['Distance-based', 'Free energy-based', 'Our classifier', 'Our regressor']
efuncs = [roc_auc_score, average_precision_score]
elabels = ['auROC', 'auPRC']
for col, plabel in zip(pcols, plabels):
    for func, elabel in zip(efuncs, elabels):
        pos_scores = truetbl[col].tolist()
        neg_scores = falsetbl[col].tolist()
        y_true = np.array([1]*len(pos_scores) + [0]*len(neg_scores)) 
        y_score = np.concatenate([pos_scores, neg_scores])
        score = func(y_true, y_score)
        scores[plabel][elabel] = max(1-score, score)
scores = pd.DataFrame(scores).T
scores.round(3)

In [ ]:
#scores.to_csv('/home/jupyter/ADAPT_PCR_share/safe/design/pipeline/origene_primers.eval.csv',index=False)
scores.to_csv('/home/jupyter/ADAPT_PCR_share/safe/design/pipeline/test_set.eval.csv')

In [ ]:
cols = ['target_seq', 'f_name', 'f_seq', 'f_start', 'f_end', 'f_len', 'f_Tm',
       'f_GC', 'f_indel', 'f_mm', 'pseq_f', 'tseq_f', 'r_name', 'r_seq',
       'r_start', 'r_end', 'r_len', 'r_Tm', 'r_GC', 'r_indel', 'r_mm',
       'pseq_r', 'tseq_r', 'prod_len', 'prod_Tm', 'score', 'binary']
test_df.columns = cols
start = time.time()
seq_input = one_hot_encode_pbs_gap_parallel(test_df)
fea_input = scaler.transform(test_df[FEATS])
labels = np.array([0]*len(test_df))
dataset = PcrDataset(seq_input, fea_input, labels)
loader = DataLoader(dataset, batch_size=64, shuffle=False)
runtime = (time.time() - start)
print('Run time (encoding): %.3f sec' % runtime)

In [ ]:
start = time.time()
predict_cls, predict_reg = [], []
with torch.no_grad():
    for seq_in, fea_in, lab in tqdm(loader, desc='Infering'):
        seq_in, fea_in, lab = seq_in.to(device).float(), fea_in.to(device).float(), lab.to(device).float()
        out_cls = classifier(seq_in)
        out_reg = model(fea_in, seq_in)
        if len(seq_in)==1:
            predict_cls.append(np.array([out_cls.squeeze().detach().cpu().numpy()]))
            predict_reg.append(np.array([out_reg.squeeze().detach().cpu().numpy()]))
        else:
            predict_cls.append(out_cls.squeeze().detach().cpu().numpy())
            predict_reg.append(out_reg.squeeze().detach().cpu().numpy())
    predict_cls = np.concatenate(predict_cls)
    predict_reg = np.concatenate(predict_reg)
test_df.loc[:,'classifier'] = predict_cls
test_df.loc[:,'regressor'] = predict_reg

In [ ]:
def get_pair_dg(row):
    row['f_dg'] = get_dg_vienna(row['pseq_f'], reverse_complement_dna(row['tseq_f']))
    row['r_dg'] = get_dg_vienna(row['pseq_r'], reverse_complement_dna(row['tseq_r']))
    return row

def get_pair_dg_parallel(df):
    rows = df.to_dict('records')
    with Pool(processes=cpu_count()) as pool:
        results = pool.map(get_pair_dg, rows)
    return pd.DataFrame(results)

# true_dg = get_pair_dg_parallel(truetbl)
# false_dg = get_pair_dg_parallel(falsetbl)

In [ ]:
truef = '/home/jupyter/ADAPT_PCR_share/safe/design/pipeline/test_set.true.dg.csv'
falsef = '/home/jupyter/ADAPT_PCR_share/safe/design/pipeline/test_set.false.dg.csv'
#truetbl.to_csv(truef,index=False)
#falsetbl.to_csv(falsef,index=False)

In [ ]:
# truef = '/home/jupyter/ADAPT_PCR_share/safe/design/pipeline/origene_primers.true.dg.csv'
# falsef = '/home/jupyter/ADAPT_PCR_share/safe/design/pipeline/origene_primers.false.dg.csv'
# truetbl.to_csv(truef,index=False)
# falsetbl.to_csv(falsef,index=False)

## Primer3

In [ ]:
def run_primer3_check(templ, fseq, rseq, pname, path):
    inptxt = '''SEQUENCE_ID=example_check
SEQUENCE_TEMPLATE=%s
PRIMER_TASK=check_primers
PRIMER_EXPLAIN_FLAG=1
SEQUENCE_PRIMER=%s
SEQUENCE_PRIMER_REVCOMP=%s
PRIMER_MIN_SIZE=15
PRIMER_OPT_SIZE=20
PRIMER_MAX_SIZE=27
PRIMER_MIN_GC=20.0
PRIMER_OPT_GC_PERCENT=50.0
PRIMER_MAX_GC=80.0
PRIMER_MIN_TM=50.0
PRIMER_OPT_TM=60.0
PRIMER_MAX_TM=72.0
PRIMER_PRODUCT_SIZE_RANGE=50-10000
PRIMER_MAX_SELF_ANY=12
PRIMER_MAX_SELF_END=8
PRIMER_PAIR_MAX_COMPL_ANY=12
PRIMER_PAIR_MAX_COMPL_END=8
PRIMER_MAX_POLY_X=5
PRIMER_MAX_HAIRPIN_TH=47.0
=''' % (templ, fseq, rseq)
    inpfile = '%s/%s.txt' % (path, pname)
    with open(inpfile, 'wt') as out:
        out.write(inptxt)
    result = !$primer3 $inpfile
    
    left_penalty = None
    right_penalty = None
    pair_penalty = None
    for line in result:
        if line.startswith("PRIMER_LEFT_0_PENALTY"):
            left_penalty = float(line.split("=")[1])
        elif line.startswith("PRIMER_RIGHT_0_PENALTY"):
            right_penalty = float(line.split("=")[1])
        elif line.startswith("PRIMER_PAIR_0_PENALTY"):
            pair_penalty = float(line.split("=")[1])

    return left_penalty, right_penalty, pair_penalty
    return res

In [ ]:
pname = 0
off = 5
fseq = truetbl.loc[pname,'pseq_f']
rseq = truetbl.loc[pname,'pseq_r']

tseq = truetbl.loc[pname,'tseq']
fst = tseq.find(fseq)
rst = fst + tseq[fst:].find(reverse_complement_dna(rseq))
templ = tseq[fst-off:rst + len(rseq)+off]

path = '/home/jupyter/ADAPT_PCR_share/safe/design/pipeline/temp'



#run_primer3_check(templ, fseq, rseq, pname, path)

In [ ]:
inptxt = '''SEQUENCE_ID=example_check
SEQUENCE_TEMPLATE=%s
PRIMER_TASK=check_primers
PRIMER_EXPLAIN_FLAG=1
SEQUENCE_PRIMER=%s
SEQUENCE_PRIMER_REVCOMP=%s
PRIMER_MIN_SIZE=15
PRIMER_OPT_SIZE=20
PRIMER_MAX_SIZE=27
PRIMER_MIN_GC=20.0
PRIMER_OPT_GC_PERCENT=50.0
PRIMER_MAX_GC=80.0
PRIMER_MIN_TM=50.0
PRIMER_OPT_TM=60.0
PRIMER_MAX_TM=72.0
PRIMER_PRODUCT_SIZE_RANGE=50-1000
PRIMER_MAX_SELF_ANY=12
PRIMER_MAX_SELF_END=8
PRIMER_PAIR_MAX_COMPL_ANY=12
PRIMER_PAIR_MAX_COMPL_END=8
PRIMER_MAX_POLY_X=5
PRIMER_MAX_HAIRPIN_TH=47.0
=''' % (templ, fseq, rseq)
inpfile = '%s/%s.txt' % (path, pname)
with open(inpfile, 'wt') as out:
    out.write(inptxt)

In [ ]:
primer3

In [ ]:
!$primer3 $inpfile

In [ ]:
print(inptxt)

In [ ]:
truetbl.head(1)

---

## Test set

In [ ]:
train_df = pd.read_csv('/home/jupyter/ADAPT_PCR_share/safe/dataset/0717_dataset_train.csv',index_col=[0,1])
valid_df = pd.read_csv('/home/jupyter/ADAPT_PCR_share/safe/dataset/0717_dataset_valid.csv',index_col=[0,1])
test_df = pd.read_csv('/home/jupyter/ADAPT_PCR_share/safe/dataset/0717_dataset_test.csv',index_col=[0,1])

In [ ]:
test_df.head()

In [ ]:
def get_pair_dg(row):
    row['f_dg'] = get_dg_vienna(row['f_seq'], reverse_complement_dna(row['f_tenc'].replace('-','').upper()))
    row['r_dg'] = get_dg_vienna(row['r_seq'], reverse_complement_dna(row['r_tenc'].replace('-','').upper()))
    return row
test_dgs = get_pair_dg_parallel(test_df)

In [ ]:
dgcols = ['f_dg','r_dg']
test_dgs['dg'] = test_dgs[dgcols].max(axis=1)

mmcols = ['f_mm','f_indel','r_mm','r_indel']
test_dgs['mm'] = test_dgs[mmcols].sum(axis=1)
test_dgs.head(1)

In [ ]:
from scipy.stats import spearmanr
from sklearn.metrics import r2_score, mean_squared_error

truet = test_df[test_df['binary']==1]
truedg = test_dgs[test_dgs['binary']==1]
print(spearmanr(truedg['mm'], truedg['score']), mean_squared_error(truedg['mm'], truedg['score']))
print(spearmanr(truedg['dg'], truedg['score']), mean_squared_error(truedg['dg'], truedg['score']))
print(spearmanr(truet['regressor'], truet['score']), mean_squared_error(truet['regressor'], truet['score']))

In [ ]:
from scipy.stats import spearmanr
from sklearn.metrics import r2_score, mean_squared_error

truet = test_df[test_df['binary']==1]
truedg = test_dgs[test_dgs['binary']==1]
print(spearmanr(truedg['mm'], truedg['score']), 
      mean_squared_error(truedg['mm'], truedg['score'])/np.var(truedg['mm']))
print(spearmanr(truedg['dg'], truedg['score']),
      mean_squared_error(truedg['dg'], truedg['score'])/np.var(truedg['dg']))
print(spearmanr(truet['regressor'], truet['score']), 
      mean_squared_error(truet['regressor'],truet['score'])/np.var(truet['regressor']))

In [ ]:
mean_squared_error?

In [ ]:
truet = test_df[test_df['binary']==1]
falset = test_df[test_df['binary']==0]

In [ ]:
mmcols = ['f_mm','f_indel','r_mm','r_indel']
pos_scores = truet[mmcols].sum(axis=1).tolist()
neg_scores = falset[mmcols].sum(axis=1).tolist()

y_true = np.array([0]*len(pos_scores) + [1]*len(neg_scores)) 
y_scores = np.concatenate([pos_scores, neg_scores])
auroc = roc_auc_score(y_true, y_scores)
print("AUROC:", auroc)

In [ ]:
def get_pair_dg(row):
    row['f_dg'] = get_dg_vienna(row['f_seq'], reverse_complement_dna(row['f_tenc'].replace('-','').upper()))
    row['r_dg'] = get_dg_vienna(row['r_seq'], reverse_complement_dna(row['r_tenc'].replace('-','').upper()))
    return row

truet_dg = get_pair_dg_parallel(truet)
falset_dg = get_pair_dg_parallel(falset)

In [ ]:
dgcols = ['f_dg','r_dg']
pos_scores = truet_dg[dgcols].max(axis=1).tolist()
neg_scores = falset_dg[dgcols].max(axis=1).tolist()

y_true = np.array([1]*len(pos_scores) + [0]*len(neg_scores)) 
y_scores = np.concatenate([pos_scores, neg_scores])
auroc = roc_auc_score(y_true, y_scores)
print("AUROC:", max(auroc, 1-auroc))

# Kayama et al. 2021

In [ ]:
# prif = '/home/jupyter/ADAPT_PCR_share/safe/design/pipeline/kayama_2021/primers_kayama_2021.csv'
# primers = pd.read_csv(prif, header=1, names=['name','seq'])
# primers = {i:row['seq'].strip().replace(' ','') for i,row in primers.iterrows()}
prif = '/home/jupyter/ADAPT_PCR_share/safe/design/pipeline/kayama_2021/primers.fa'
primerseqs = {s.id:str(s.seq) for s in SeqIO.parse(prif,'fasta')}
# with open(prif,'wt') as out:
#     for i in range(72):
#         out.write('>%i_f\n%s\n>%i_r\n%s\n'%(i+1,primers[2*i],i+1,primers[2*i+1]))
targetf =  '/home/jupyter/ADAPT_PCR_share/safe/design/pipeline/kayama_2021/targets.fa'
targetseqs = {s.id:str(s.seq) for s in SeqIO.parse(targetf,'fasta')}
print(len(primerseqs), len(targetseqs))

In [ ]:
def find_match(seq, ref, dcut=10):
    if len(ref) < len(seq):
        print('x')
        return (-1, 999) 
    if seq in ref:
        return (ref.find(seq), 0)
    sortvs = sorted([(distance(seq, ref[i:i+len(seq)]), i) for i in range(len(ref)-len(seq))])
    d, st = sortvs[0]
    return (st, d) if d <= dcut else (-1, 999)

dists = defaultdict(lambda: defaultdict(int))
for pname, pseq in primerseqs.items():
    pid, ori = pname.split('_')
    for tname, tseq in targetseqs.items():
        if ori=='f':
            st, d = find_match(pseq, tseq)
        else:
            st, d = find_match(reverse_complement_dna(pseq), tseq)
        dists[tname][pid] += d
        
dists = pd.DataFrame(dists)

In [ ]:
labelf = '/home/jupyter/ADAPT_PCR_share/safe/design/pipeline/kayama_2021/labels_kayama_2021.csv'
labtbl = pd.read_csv(labelf, index_col=0)
labtbl.head()

In [ ]:
xs = np.array(labtbl).flatten()
ys = np.array(dists).flatten()

ys0 = [y for x,y in zip(xs,ys) if x==0]
ys1 = [y for x,y in zip(xs,ys) if x==1]

bp = plt.boxplot([ys0,ys1], sym='+', widths=.8)
plt.xticks([1,2],['No PCR','PCR'])
plt.ylabel('Primer-target distance')